In [14]:
import pandas as pd, os, inflection, numpy as np

In [15]:
mdb_to_excel_file_loc = r'C:\Users\E048374\Documents\RawNav'

stop_file = os.path.join(mdb_to_excel_file_loc, '082719-201718 Stop.xlsx')
stop_dat = pd.read_excel(stop_file)
stop_dat = stop_dat.dropna(axis=1)
stop_dat.columns = [inflection.underscore(col_nm) for col_nm in stop_dat.columns]
stop_dat.rename(columns = {'longitude':'stop_lon', 
                           'latitude':'stop_lat',
                           'heading':'stop_heading'}, inplace = True)

In [16]:
pattern_file = os.path.join(mdb_to_excel_file_loc, '082719-201718 Pattern.xlsx')
pattern_dat = pd.read_excel(pattern_file)
pattern_dat = pattern_dat[['PatternID','TARoute','PatternName','Direction',
                           'Distance','CDRoute','CDVariation','PatternDestination',
                           'RouteText','RouteKey','PatternDestination2','RouteText2',
                           'Direction2','PatternName2','TARoute2','PubRouteDir','PatternNotes',
                           'DirectionID']]

In [17]:
pattern_dat.columns = [inflection.underscore(col_nm) for col_nm in pattern_dat.columns]
pattern_dat.rename(columns={'distance':'trip_length',
                            'cd_route':'route',
                            'cd_variation':'pattern'},inplace=True)

pattern_detail_file = os.path.join(mdb_to_excel_file_loc, '082719-201718 PatternDetail.xlsx')
pattern_detail_dat = pd.read_excel(pattern_detail_file)
pattern_detail_dat = pattern_detail_dat.dropna(axis=1)
pattern_detail_dat = pattern_detail_dat.drop(columns=['SortOrder', 'GeoPathID'])
pattern_detail_dat.columns = [inflection.underscore(col_nm) for col_nm in pattern_detail_dat.columns]
pattern_detail_dat.rename(columns={'distance':'dist_from_previous_stop'},inplace=True)

In [18]:
q_jump_route_list = ['S4','52'] ## update
pattern_q_jump_route_dat = pattern_dat.query('route in @q_jump_route_list')
set(pattern_q_jump_route_dat.route.unique()) - set(q_jump_route_list)

set()

In [19]:
pattern_q_jump_route_dat.head()

,pattern_id,ta_route,pattern_name,direction,trip_length,route,pattern,pattern_destination,route_text,route_key,pattern_destination2,route_text2,direction2,pattern_name2,ta_route2,pub_route_dir,pattern_notes,direction_id
5,806,S4,[S4]SILVER SPRNG - FED TRIANGLE,SOUTH,41437,S4,1,SOUTH to FEDERAL TRIANGLE,16th Street Line,535,NaN,NaN,NaN,NaN,NaN,SOUTH,NaN,1.0
6,808,S4,[S4]FED TRIANGLE - SILVER SPRNG,NORTH,43108,S4,2,NORTH to SILVER SPRING STATION,16th Street Line,547,NaN,NaN,NaN,NaN,NaN,NORTH,NaN,0.0
74,1306,52,[52]L ENFNT PLAZA - 14TH+COLORADO,NORTH,34532,52,4,NORTH to 14TH & COLORADO,14th Street Line,585,NaN,NaN,NaN,NaN,NaN,NORTH,NaN,0.0
75,1307,52,[52]L ENFNT PLAZA - TAKOMA STATION,NORTH,47617,52,2,NORTH to TAKOMA STATION,14th Street Line,580,NaN,NaN,NaN,NaN,NaN,NORTH,NaN,0.0
76,1311,52,[52]TAKOMA STA - L ENFANT PLAZA STA,SOUTH,47497,52,3,SOUTH to L'ENFANT PLAZA STATION,14th Street Line,718,NaN,NaN,NaN,NaN,NaN,SOUTH,NaN,1.0


In [20]:
pattern_pattern_detail_stop_q_jump_route_dat = \
    pattern_q_jump_route_dat.merge(pattern_detail_dat,on='pattern_id',how='left')\
    .merge(stop_dat,on='geo_id',how='left')

pattern_pattern_detail_stop_q_jump_route_dat.\
    sort_values(by=['route','pattern','order'],inplace=True)

mask_nan_latlong = pattern_pattern_detail_stop_q_jump_route_dat[['stop_lat', 'stop_lon']].isna().all(axis=1)
assert_stop_sort_order_zero_has_nan_latlong = \
    sum(pattern_pattern_detail_stop_q_jump_route_dat[mask_nan_latlong].stop_sort_order-0)
assert(assert_stop_sort_order_zero_has_nan_latlong==0)

no_nan_pattern_pattern_detail_stop_q_jump_route_dat =\
    pattern_pattern_detail_stop_q_jump_route_dat[~mask_nan_latlong]

no_nan_pattern_pattern_detail_stop_q_jump_route_dat = \
    no_nan_pattern_pattern_detail_stop_q_jump_route_dat.dropna(axis=1)

assert(0== sum(~ no_nan_pattern_pattern_detail_stop_q_jump_route_dat.
               eval('''direction==pub_route_dir& route==ta_route''')))
no_nan_pattern_pattern_detail_stop_q_jump_route_dat.drop(columns=['pub_route_dir','ta_route'],inplace=True)

assert(0== np.sum(no_nan_pattern_pattern_detail_stop_q_jump_route_dat.isna().values))

In [21]:
save_file = os.path.join(mdb_to_excel_file_loc,'wmata_schedule_data_q_jump_routes.csv')
no_nan_pattern_pattern_detail_stop_q_jump_route_dat.to_csv(save_file)

In [22]:
no_nan_pattern_pattern_detail_stop_q_jump_route_dat.iloc[0]

pattern_id                                               1315
pattern_name               [52]14TH+COLORADO - L ENFANT PLAZA
direction                                               SOUTH
trip_length                                             35165
route                                                      52
pattern                                                     1
pattern_destination           SOUTH to L'ENFANT PLAZA STATION
route_text                                   14th Street Line
route_key                                                 154
direction_id                                                1
geo_id                                                   8676
stop_id                                                  6192
dist_from_previous_stop                                     0
order                                                       2
stop_sort_order                                             1
geo_description                  COLORADO AVE NW + 14TH ST NW
ta_geo_i

In [23]:
no_nan_pattern_pattern_detail_stop_q_jump_route_dat.route.unique()

array(['52', 'S4'], dtype=object)